In [2]:
import httpx

In [4]:
url = 'http://localhost:8000/import/loc/works?uri=http%3A%2F%2Fid.loc.gov%2Fresources%2Fworks%2F1893399'
response = httpx.get(url)
response

<Response [200 OK]>

In [5]:
from pydantic import BaseModel
from typing import Optional
from api.src.schemas.metadata.bibframe.adminMetadata import AdminMetadata
from api.src.schemas.metadata.bibframe.classification import Classification
from api.src.schemas.metadata.bibframe.contribution import Contribution
from api.src.schemas.metadata.bibframe.element import Element
from api.src.schemas.metadata.bibframe.title import Title

class Work(BaseModel):
    adminMetadata: AdminMetadata
    type: list[str]
    content: list[Element]
    language: list[Element]
    title: Title
    classification: Optional[Classification] = None
    contribution: Optional[list[Contribution]] = None
    subject: Optional[list[Element]] = None   
    genreForm: Optional[list[Element]] = None
    note: Optional[str] = None
    summary: Optional[str] = None
    tableOfContents: Optional[str] = None
    supplementaryContent: Optional[list[Element]] = None
    illustrativeContent: Optional[list[Element]] = None
    intendedAudience: Optional[list[Element]] = None
    geographicCoverage: Optional[list[Element]] = None

In [16]:
obj = response.json()
obj['classification'] = {
    "type": "cdd",
    "classificationPortion": "542.5",
    "itemPortion": "P542c"
}

In [18]:
obj['classification']

{'type': 'cdd', 'classificationPortion': '542.5', 'itemPortion': 'P542c'}

In [40]:
request = Work(**obj)
request

Work(adminMetadata=AdminMetadata(encodingLevel='menclvl:f', assigner='http://id.loc.gov/vocabulary/organizations/brmninpa', creationDate=datetime.date(2023, 7, 18), descriptionConventions='http://id.loc.gov/vocabulary/descriptionConventions/isbd', descriptionModifier='http://id.loc.gov/vocabulary/organizations/brmninpa', descriptionLanguage='http://id.loc.gov/vocabulary/languages/por', generationProcess='BiblioKeia v.1', generationDate=datetime.datetime(2023, 7, 18, 14, 43, 41, 100189), identifiedBy=[IdentifiedBy(type='Local', assigner='http://id.loc.gov/vocabulary/organizations/dlc', value='1893399')], status=Status(value='mstatus:new', label='novo')), type=['Work', 'Text'], content=[Element(label='text', lang=None, uri='http://id.loc.gov/vocabulary/contentTypes/txt', type=['http://id.loc.gov/ontologies/bibframe/Content'])], language=[Element(label='English', lang='en', uri='http://id.loc.gov/vocabulary/languages/eng', type=['http://id.loc.gov/ontologies/bibframe/Language'])], title=T

In [34]:
request.contribution[0].agent = 'https://bibliokeia.com/authorities/PersonalName/bka-4'

In [41]:
from api.src.function.bibframe.Work.graphWork import MakeGraphWork


G = MakeGraphWork(request, "bkc-3")

In [43]:
print(G)

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
    PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
    PREFIX bflc: <http://id.loc.gov/ontologies/bflc/> 
    PREFIX bkw: <https://bibliokeia.com/resources/work/> 
    PREFIX menclvl: <http://id.loc.gov/vocabulary/menclvl/>
    PREFIX mstatus: <http://id.loc.gov/vocabulary/mstatus/>
    PREFIX contentTypes: <http://id.loc.gov/vocabulary/contentTypes/>
    PREFIX relators: <http://id.loc.gov/vocabulary/relators/>
    PREFIX genreForms: <http://id.loc.gov/authorities/genreForms/>
    PREFIX msupplcont: <http://id.loc.gov/vocabulary/msupplcont/>
    PREFIX millus: <http://id.loc.gov/vocabulary/millus/>    
        
    INSERT DATA {
        GRAPH bkw:bkc-3
        {
                bkw:bkc-3 a bf:Work, bf:Text  ;
                bf:adminMetadata [ a bf:AdminMetadata 

In [2]:
from api.src.function.bibframe.bf_classification import GetClassification
# from api.src.function.bibframe.bf_contribution import GetContribution
from api.src.function.bibframe.bf_Uri import GetUriBF
from api.src.function.bibframe.bf_Literal import GetLiteral
from api.src.function.bibframe.bf_title import GetTitle
from api.src.function.bibframe.bf_type import GetType

from api.src.schemas.settings import Settings
from pyfuseki import FusekiUpdate

settings = Settings()

collectionUpdate = FusekiUpdate(f'{settings.url}:3030', 'collection')

def GetContribution(graph, uri, obj):
    
  q = f"""PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
                PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
                PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
                SELECT ?type ?role ?agent ?label
                  WHERE {{ 
                    <{uri}> bf:contribution ?contribution .
                      ?contribution rdf:type ?type .
                      ?contribution bf:role ?role .
                      ?contribution bf:agent ?agent .
                      ?agent rdfs:label ?label
                      FILTER ( ?type != bf:Contribution )
                    }}"""
  response = graph.query(q)
  bindings = response.bindings
  if len(bindings) > 0:
      contributions = list()
      for binding in bindings:
         c = {'type': [binding.get('type').toPython()],
          'agent': binding.get('agent').toPython(),
          'label': binding.get('label').toPython(),
          'role': binding.get('role').toPython()}
         contributions.append(c)
      obj['contribution'] = contributions
      
  return obj

def ParserWork(graph, uri):
    identifier = uri.split("/")[-1]
    types = GetType(graph, uri)
    title = GetTitle(graph, uri)

    obj = {
    'adminMetadata': {
        'identifiedBy': [ 
            {
                "type": "Local",
            "assigner": "http://id.loc.gov/vocabulary/organizations/dlc",
            "value": identifier
            }
        ]
    },
    "type": types,
    'title': title
    }
    
    # Classification
    obj = GetClassification(graph, uri, obj)
    # Content
    # obj = GetContent(graph, uri, obj)
    # Contribution
    obj = GetContribution(graph, uri, obj)
    # Content
    obj = GetUriBF(graph, uri, 'content', obj)
    # GenreForm
    obj = GetUriBF(graph, uri, 'genreForm', obj)
    # illustrativeContent
    obj = GetUriBF(graph, uri, 'illustrativeContent', obj)
    # intendedAudience
    obj = GetUriBF(graph, uri, 'intendedAudience', obj)
    # language
    obj = GetUriBF(graph, uri, 'language', obj)
    # subject
    obj = GetUriBF(graph, uri, 'subject', obj)
    # supplementaryContent
    obj = GetUriBF(graph, uri, 'supplementaryContent', obj)
    # geographicCoverage
    obj = GetUriBF(graph, uri, 'geographicCoverage', obj)
    # summary
    obj = GetLiteral(graph, uri, 'summary', obj)
    # tableOfContents
    obj = GetLiteral(graph, uri, 'tableOfContents', obj)

    # response = Work(**obj)
    
    return obj


In [11]:
obj = ParserWork(graph, uri) 
obj

{'adminMetadata': {'identifiedBy': [{'type': 'Local',
    'assigner': 'http://id.loc.gov/vocabulary/organizations/dlc',
    'value': '3227597'}]},
 'type': ['Work', 'Text'],
 'title': {'mainTitle': 'Physiological plant ecology', 'subtitle': None},
 'classification': {'type': 'ClassificationDdc',
  'classificationPortion': '581.5',
  'itemPortion': None,
  'edition': 'full'},
 'contribution': [{'type': ['http://id.loc.gov/ontologies/bflc/PrimaryContribution'],
   'agent': 'http://id.loc.gov/rwo/agents/n79125202',
   'label': 'Larcher, W. (Walter), 1929-',
   'role': 'http://id.loc.gov/vocabulary/relators/ctb'}],
 'content': [{'label': 'text',
   'lang': None,
   'uri': 'http://id.loc.gov/vocabulary/contentTypes/txt',
   'type': ['http://id.loc.gov/ontologies/bibframe/Content']}],
 'illustrativeContent': [{'label': 'Illustrations',
   'lang': None,
   'uri': 'http://id.loc.gov/vocabulary/millus/ill',
   'type': ['http://id.loc.gov/ontologies/bibframe/Illustration']}],
 'language': [{'lab

In [3]:
import httpx

url = 'http://localhost:8000/import/loc/works?uri=http%3A%2F%2Fid.loc.gov%2Fresources%2Fworks%2F3227597'
response = httpx.get(url)
response

<Response [200 OK]>

In [5]:
obj = response.json()

In [6]:
obj['contribution'][0]['agent'] = 'https://bibliokeia.com/authorities/PersonalName/bka-2'
obj['subject'][0]['uri'] = 'https://bibliokeia.com/authorities/Topic/bka-1'

In [7]:
request = Work(**obj)
request

Work(adminMetadata=AdminMetadata(encodingLevel='menclvl:f', assigner='http://id.loc.gov/vocabulary/organizations/brmninpa', creationDate=datetime.date(2023, 7, 18), descriptionConventions='http://id.loc.gov/vocabulary/descriptionConventions/isbd', descriptionModifier='http://id.loc.gov/vocabulary/organizations/brmninpa', descriptionLanguage='http://id.loc.gov/vocabulary/languages/por', generationProcess='BiblioKeia v.1', generationDate=datetime.datetime(2023, 7, 18, 13, 34, 19, 934622), identifiedBy=[IdentifiedBy(type='Local', assigner='http://id.loc.gov/vocabulary/organizations/dlc', value='3227597')], status=Status(value='mstatus:new', label='novo')), type=['Work', 'Text'], content=[Element(label='text', lang=None, uri='http://id.loc.gov/vocabulary/contentTypes/txt', type=['http://id.loc.gov/ontologies/bibframe/Content'])], language=[Element(label='English', lang='en', uri='http://id.loc.gov/vocabulary/languages/eng', type=['http://id.loc.gov/ontologies/bibframe/Language', 'http://ww

In [9]:
from pyfuseki import FusekiUpdate
from api.src.schemas.settings import Settings
import pysolr  

settings = Settings()

authorityUpdate = FusekiUpdate(f'{settings.url}:3030', 'authority')
#SOLR
solr = pysolr.Solr(f'{settings.url}:8983/solr/authority/', timeout=10)

def SubjectOf(request, collection, id):

    subjectOf = """PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
                INSERT DATA
                {{ GRAPH <{subject}> {{ 
                    <{subject}>  bf:subjectOf <https://bibliokeia.com/resources/{collection}/{id}> }} }} ; """
 
    for subject in request.subject:
        # Jena
        up = subjectOf.format(subject=subject.uri, collection=collection, id=id)
        response = authorityUpdate.run_sparql(up)
        print("UP JENA:", response.convert())

        # Solr
        sID = subject.uri.split('/')[-1]
        doc = {
        "id": sID,
        "subjectOf": { "add" : {
            "id": f"{sID}/subjectOf/{id}",
            'label': request.title.mainTitle,
            'uri': f'https://bibliokeia.com/resources/{collection}/{id}'
        } }}    
    
        responseSolr =  solr.add([doc], commit=True)
        print("UP SOLR:", responseSolr)

In [10]:
id = 'bkc-1'
SubjectOf(request, 'work', id)

UP JENA: {'statusCode': 200, 'message': 'Update succeeded'}
UP SOLR: {
  "responseHeader":{
    "status":0,
    "QTime":223}}



In [ ]:
from api.src.function.bibframe.Work.contributionOf import ContributionOf


ContributionOf(request, 'work', id)

In [13]:
from pyfuseki import FusekiUpdate
from api.src.schemas.settings import Settings
import pysolr  

settings = Settings()

authorityUpdate = FusekiUpdate(f'{settings.url}:3030', 'authority')
#SOLR
solr = pysolr.Solr(f'{settings.url}:8983/solr/authority/', timeout=10)

def ContributionOf(request, collection, id):

    contributionOf = """PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>

                    INSERT DATA
                        {{ GRAPH <{contribution}> {{ 
                            <{contribution}>  bf:contributionOf <https://bibliokeia.com/resources/{collection}/{id}> }} }} ; """
    for contribution in request.contribution:
        # Jena
        up = contributionOf.format(contribution=contribution.agent, collection=collection, id=id)
        response = authorityUpdate.run_sparql(up)
        print(response.convert())

        # Solr
        cID = contribution.agent.split('/')[-1]
        doc = {
        "id": cID,
        "contributionOf": { "add" : {
            "id": f"{cID}/contributionOf/{id}",
            'label': request.title.mainTitle,
            'uri': f'https://bibliokeia.com/resources/{collection}/{id}'
        } }} 
        responseSolr =  solr.add([doc], commit=True)
        print("UP SOLR:", responseSolr)

In [14]:
ContributionOf(request, 'work', id)

{'statusCode': 200, 'message': 'Update succeeded'}
UP SOLR: {
  "responseHeader":{
    "status":0,
    "QTime":180}}



In [15]:
import pysolr 

#SOLR
solr = pysolr.Solr('http://localhost:8983/solr/collection/', timeout=10)

def DocInstance(request, id):

    workID = request.instanceOf.uri.split("/")[-1]
    uri = f'https://bibliokeia.com/resources/instance/{id}'

    doc = {
        "id": id,
        "type": request.type,
        "mainTitle": request.title.mainTitle,
        "subtitle": request.title.subtitle,
        "carrier": [i.label for i in request.carrier],
        "dimensions": request.dimensions,
        "extent": request.extent.label if request.extent else None,
        "issuance": [i.label for i in request.issuance],
        "media": [i.label for i in request.media],
        "provisionActivityAgent": request.provisionActivity.agent,
        "provisionActivityDate": request.provisionActivity.date,
        "provisionActivityPlace": request.provisionActivity.place,      
        "serie": request.seriesStatement,
        "instanceOf": {'id': f'{id}/instanceOf/{workID}', 'uri': request.instanceOf.uri, 'label': request.instanceOf.label} 
        }
    
    work = {
        "id": workID,
         "hasInstance": {"add": {'id': f'{workID}/hasInstance/{id}', 'uri': uri, 'label': request.title.mainTitle} }
    }
    print("hasInstance", work)

    response = solr.add([doc, work], commit=True)
    return response

In [17]:
obj = {
  "adminMetadata": {
    "encodingLevel": "menclvl:f",
    "assigner": "http://id.loc.gov/vocabulary/organizations/brmninpa",
    "creationDate": "2023-07-18",
    "descriptionConventions": "http://id.loc.gov/vocabulary/descriptionConventions/isbd",
    "descriptionModifier": "http://id.loc.gov/vocabulary/organizations/brmninpa",
    "descriptionLanguage": "http://id.loc.gov/vocabulary/languages/por",
    "generationProcess": "BiblioKeia v.1",
    "generationDate": "2023-07-18T14:07:28.790958",
    "identifiedBy": [
      {
        "type": "Local",
        "assigner": "http://id.loc.gov/vocabulary/organizations/dlc",
        "value": "3227597"
      }
    ],
    "status": {
      "value": "mstatus:new",
      "label": "novo"
    }
  },
  "type": [
    "Instance",
    "Print"
  ],
  "title": {
    "type": "bf:Title",
    "mainTitle": "Physiological plant ecology"
  },
  "carrier": [
    {
      "label": "volume",
      "uri": "http://id.loc.gov/vocabulary/carriers/nc",
      "type": [
        "http://id.loc.gov/ontologies/bibframe/Carrier"
      ]
    }
  ],
  "dimensions": "23 cm.",
  "extent": {
    "label": "xvii, 303 p.",
    "type": "http://id.loc.gov/ontologies/bibframe/Extent"
  },
  "instanceOf": {
    "label": "Larcher, W. (Walter), 1929- Physiological plant ecology",
    "uri": "https://bibliokeia.com/resources/work/bkc-1",
    "type": [
      "http://id.loc.gov/ontologies/bibframe/Work"
    ]
  },
  "issuance": [
    {
      "label": "single unit",
      "uri": "http://id.loc.gov/vocabulary/issuance/mono",
      "type": [
        "http://id.loc.gov/ontologies/bibframe/Issuance"
      ]
    }
  ],
  "media": [
    {
      "label": "unmediated",
      "uri": "http://id.loc.gov/vocabulary/mediaTypes/n",
      "type": [
        "http://id.loc.gov/ontologies/bibframe/Media"
      ]
    }
  ],
  "provisionActivity": {
    "date": "\"1980\"",
    "place": "Germany"
  },
  "responsibilityStatement": "W. Larcher ; translated by M. A. Biederman-Thorson"
}

In [18]:
from pydantic import BaseModel
from typing import Optional

class ProvisionActivity(BaseModel):
    agent: Optional[str] = None
    date: str
    place: str

In [22]:
class Value(BaseModel):
    label: str
    type: str

class Element(BaseModel):
    label: str
    lang: Optional[str] = None
    uri: str
    type: list[str]

class Instance(BaseModel):
    adminMetadata: AdminMetadata
    type: list[str]
    title: Title
    carrier: list[Element]
    copyrightDate: Optional[str] = None
    dimensions: Optional[str] = None
    extent: Optional[Value] = None
    instanceOf: Element
    issuance: Optional[list[Element]] = None
    media: Optional[list[Element]] = None
    provisionActivity: ProvisionActivity
    provisionActivityStatement: Optional[str] = None
    responsibilityStatement: Optional[str] = None
    seriesStatement: Optional[str] = None

In [23]:
request = Instance(**obj)
request

Instance(adminMetadata=AdminMetadata(encodingLevel='menclvl:f', assigner='http://id.loc.gov/vocabulary/organizations/brmninpa', creationDate=datetime.date(2023, 7, 18), descriptionConventions='http://id.loc.gov/vocabulary/descriptionConventions/isbd', descriptionModifier='http://id.loc.gov/vocabulary/organizations/brmninpa', descriptionLanguage='http://id.loc.gov/vocabulary/languages/por', generationProcess='BiblioKeia v.1', generationDate=datetime.datetime(2023, 7, 18, 14, 7, 28, 790958), identifiedBy=[IdentifiedBy(type='Local', assigner='http://id.loc.gov/vocabulary/organizations/dlc', value='3227597')], status=Status(value='mstatus:new', label='novo')), type=['Instance', 'Print'], title=Title(type='bf:Title', mainTitle='Physiological plant ecology', subtitle=None, label=None), carrier=[Element(label='volume', lang=None, uri='http://id.loc.gov/vocabulary/carriers/nc', type=['http://id.loc.gov/ontologies/bibframe/Carrier'])], copyrightDate=None, dimensions='23 cm.', extent=Value(label

In [24]:
request.instanceOf

Element(label='Larcher, W. (Walter), 1929- Physiological plant ecology', lang=None, uri='https://bibliokeia.com/resources/work/bkc-1', type=['http://id.loc.gov/ontologies/bibframe/Work'])

In [25]:
responseSolr = DocInstance(request, 'bkc-2')
responseSolr

hasInstance {'id': 'bkc-1', 'hasInstance': {'add': {'id': 'bkc-1/hasInstance/bkc-2', 'uri': 'https://bibliokeia.com/resources/instance/bkc-2', 'label': 'Physiological plant ecology'}}}


'{\n  "responseHeader":{\n    "status":0,\n    "QTime":428}}\n'

In [26]:
import httpx
url = 'http://localhost:8000/import/loc/works?uri=http%3A%2F%2Fid.loc.gov%2Fresources%2Fworks%2F1893399'
response = httpx.get(url)
response

<Response [200 OK]>

In [28]:
obj = response.json()

In [31]:
obj['subject'][0]['uri'] = 'https://bibliokeia.com/authorities/Topic/bka-3'

In [35]:
obj['contribution'][0]['agent'] = 'https://bibliokeia.com/authorities/PersonalName/bka-4'

In [44]:
from pyfuseki import FusekiQuery

from api.src.schemas.settings import Settings

settings = Settings()

def GenerateId():

    collection = FusekiQuery(f'{settings.url}:3030', 'collection')

    sparql = """PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
            SELECT ?g
            {graph ?g {?s bf:generationDate ?o}} 
            ORDER BY DESC(?o)
            LIMIT 1"""
    
    response = collection.run_sparql(sparql)
    response = response.convert()
    bindings = response['results']['bindings']

    if len(bindings) == 0:
        return { 'id': "bkc-1" }
    else:
        value = bindings[0]['g']['value']
        bk = int(value.split("-")[1]) + 1
        return { 'id': "bkc-"+str(bk) } 

In [45]:
GenerateId()

{'id': 'bkc-6'}